### Setup

In [7]:
# Main
import numpy as np
import pandas as pd

# Unit root
from quantileADF import QADF
from report import countryReport, reportCountries

# Utilities
import warnings
from statsmodels.tools.sm_exceptions import IterationLimitWarning
warnings.simplefilter('ignore', IterationLimitWarning)
pd.set_option('display.max_rows', 500)

# Setting up Urls
repoUrl = 'https://raw.githubusercontent.com/vlad-yeghiazaryan/QADF/master/data/'
datasetUrl = repoUrl + 'dataset.csv'
broadREERsUrl = repoUrl + 'broadREERs.csv'
narrowREERsUrl = repoUrl + 'narrowREERs.csv'

In [8]:
broadREERs = pd.read_csv(broadREERsUrl, index_col=0, parse_dates=True)
narrowREERs = pd.read_csv(narrowREERsUrl, index_col=0, parse_dates=True)

# Extracting country information
countries = ['Armenia', 'Belarus', 'Kazakhstan', 'Kyrgyz Republic',
             'Moldova', 'Russian Federation', 'Tajikistan', 'Ukraine']

startDate = '2000-01-01'
startDateWithoutTaj = '1995-01-01'

dataNarrow =  narrowREERs[countries].loc[startDate:]
dataNarrow.index.freq = 'MS'

data = broadREERs[countries].loc[startDate:]
data.index.freq = 'MS'

data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2000-01-01 to 2020-11-01
Freq: MS
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Armenia             251 non-null    float64
 1   Belarus             251 non-null    float64
 2   Kazakhstan          251 non-null    float64
 3   Kyrgyz Republic     251 non-null    float64
 4   Moldova             251 non-null    float64
 5   Russian Federation  251 non-null    float64
 6   Tajikistan          251 non-null    float64
 7   Ukraine             251 non-null    float64
dtypes: float64(8)
memory usage: 17.6 KB


In [9]:
# renormalize REERs at 2000q1 = 100
data = 100*data/data.iloc[0,:]

# Changing data
dataLogs = data.apply(np.log, axis=1)
dataPtC = dataLogs.diff()[1:]

### Report

In [4]:
data.head(2)

,Armenia,Belarus,Kazakhstan,Kyrgyz Republic,Moldova,Russian Federation,Tajikistan,Ukraine
Date,,,,,,,,
2000-01-01,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000
2000-02-01,101.346867,90.353012,100.811879,100.775559,99.74606,101.159028,99.425912,101.501266


In [5]:
levels=[0.01,0.05,0.1]
quantiles = np.arange(0.1, 1, 0.1)
drop = ['δ²', 'Lags', 'ρ₁(OLS)', 'CV1%', 'CV5%', 'CV10%']
qadfParams =  {'model':'ct', 'pmax':12, 'ic':'AIC'}
reps = 150

In [6]:
# Broad EAEU logs
logResults, logReport = reportCountries(dataLogs, qadfParams, quantiles, repetitions=reps, 
                                  significanceLevels=levels, dropColumns=drop)
logReport.to_excel('Outputs/logReport(C&T).xlsx', float_format="%.3f")

Starting Execution:

Armenia finished in: 146.47s
Belarus finished in: 244.96s
Kazakhstan finished in: 171.2s
Kyrgyz Republic finished in: 145.0s
Moldova finished in: 189.88s
Russian Federation finished in: 260.04s
Tajikistan finished in: 176.39s
Ukraine finished in: 167.73s

Total time spent executing: 25m, 2s


In [10]:
logReport

quantile                                          0.1              0.2  \
Countries          Variable/Quantile                                     
Armenia            α₀(τ)                   0.24(0.14)       0.21(0.08)   
                   ρ₁(τ)                  0.94(0.153)      0.95(0.08)*   
                   Half-lives (years)               1                1   
                   tₙ(τ)                -2.41(0.073)*    -3.2(0.033)**   
                   QKS                      3.2(0.18)                    
Belarus            α₀(τ)                   0.5(0.073)    0.44(0.007)**   
                   ρ₁(τ)                  0.89(0.08)*    0.9(0.007)***   
                   Half-lives (years)               0                1   
                   tₙ(τ)                 -1.68(0.107)  -2.88(0.007)***   
                   QKS                   5.54(0.0)***                    
Kazakhstan         α₀(τ)                  0.12(0.327)      0.02(0.233)   
                   ρ₁(τ)                  0.97(0.367)      0.99(0.827)   
                   Half-lives (years)               2               11   
                   tₙ(τ)                   -1.1(0.38)     -0.48(0.847)   
                   QKS                       1.9(0.6)                    
Kyrgyz Republic    α₀(τ)                  0.41(0.073)      0.23(0.133)   
                   ρ₁(τ)                  0.91(0.08)*       0.95(0.14)   
                   Half-lives (years)               1                1   
                   tₙ(τ)                  -2.39(0.16)      -2.35(0.18)   
                   QKS                    2.39(0.587)                    
Moldova            α₀(τ)                  0.27(0.127)      0.21(0.187)   
                   ρ₁(τ)                  0.94(0.133)      0.95(0.193)   
                   Half-lives (years)               1                1   
                   tₙ(τ)                -2.61(0.067)*       -2.38(0.2)   
                   QKS                     2.61(0.44)                    
Russian Federation α₀(τ)                -0.19(0.0)***    -0.17(0.0)***   
                   ρ₁(τ)                    1.04(1.0)        1.03(1.0)   
                   Half-lives (years)               ∞                ∞   
                   tₙ(τ)                  1.09(0.993)        2.13(1.0)   
                   QKS                  4.8(0.007)***                    
Tajikistan         α₀(τ)                 0.66(0.0)***     0.35(0.0)***   
                   ρ₁(τ)                 0.85(0.0)***     0.92(0.0)***   
                   Half-lives (years)               0                1   
                   tₙ(τ)               -3.41(0.013)**    -2.62(0.053)*   
                   QKS                   3.41(0.073)*                    
Ukraine            α₀(τ)                 -0.03(0.04)*       0.08(0.48)   
                   ρ₁(τ)                    1.0(0.96)      0.98(0.507)   
                   Half-lives (years)               ∞                3   
                   tₙ(τ)                   0.06(0.96)       -0.86(0.6)   
                   QKS                   3.88(0.04)**                    

quantile                                         0.3             0.4  \
Countries          Variable/Quantile                                   
Armenia            α₀(τ)                 0.09(0.467)     0.04(0.287)   
                   ρ₁(τ)                 0.98(0.433)      0.99(0.68)   
                   Half-lives (years)              3               6   
                   tₙ(τ)                 -1.32(0.48)      -0.59(0.8)   
                   QKS                                                 
Belarus            α₀(τ)               0.47(0.007)**   0.41(0.007)**   
                   ρ₁(τ)               0.9(0.007)***  0.91(0.007)***   
                   Half-lives (years)              1               1   
                   tₙ(τ)               -4.45(0.0)***   -5.26(0.0)***   
                   QKS                                                 
Kazakhstan         α₀(τ)   